In [ ]:
module fft (
    input clk,
    input rst_n,
    input wire signed [3:0] FFTInRe,
    input wire signed [3:0] FFTInIm,


    output wire signed [12:0] MDCOutUpRe,
    output wire signed [12:0] MDCOutUpIm,
    output wire signed [12:0] MDCOutDownRe,
    output wire signed [12:0] MDCOutDownIm

);
    reg [3:0] inUI_re;
    reg [3:0] inUI_im;
    reg [3:0] inLI_re;
    reg [3:0] inLI_im;

    wire [3:0] Up_out_re;
    wire [3:0] Up_out_im;
    wire [3:0] Low_out_re;
    wire [3:0] Low_out_im;

    wire [3:0] state1_shift_re;
    wire [3:0] state1_shift_im;

    wire [4:0] butter_1_up_re;
    wire [4:0] butter_1_up_im;

    wire [4:0] butter_1_l_re;
    wire [4:0] butter_1_l_im;

    wire [12:0] mul_1_up_re;
    wire [12:0] mul_1_up_im;

    wire [12:0] mul_1_l_re;
    wire [12:0] mul_1_l_im;

    wire [3:0] rom_16_counter;
    wire flag_in_com1;
    wire flag_in_com2;
    wire flag_in_com3;
    wire flag_in_com4;

    wire [8:0] rom16_re;
    wire [8:0] rom16_im;

    wire [12:0] state1_shift_2_re;
    wire [12:0] state1_shift_2_im;

    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            inUI_re <= 0;
            inUI_im <= 0;

            
        end else begin
            inLI_re <= FFTInRe;
            inLI_im <= FFTInIm;
        end
    end

    wire valid_out_rom16;
    wire flag_switch_state1_1;
    wire flag_switch_state2_1;

    controller controler_1(clk, rst_n, rom_16_counter, flag_in_com1, flag_in_com2, flag_in_com3, flag_in_com4, flag_switch_state2_1);

    commutator_input com1 (1'b1, flag_in_com1, flag_in_com2, flag_switch_state2_1, inUI_re, inUI_im, inLI_re, inLI_im, Up_out_re, Up_out_im, Low_out_re, Low_out_im);
    
    shift_16#(4) shift_1(clk, rst_n, Up_out_re, Up_out_im, state1_shift_re, state1_shift_im);

    butterfly#(5) butter_1(1'b0, state1_shift_re, state1_shift_im, Low_out_re, Low_out_im, butter_1_up_re, butter_1_up_im, butter_1_l_re, butter_1_l_im);

    ROM16 rom_16(rom_16_counter, rom16_re, rom16_im, valid_out_rom16);

    multiply mul_1(1'b0, butter_1_up_re, butter_1_up_im, rom16_re, rom16_im, mul_1_up_re, mul_1_up_im);
    multiply mul_2(1'b0, butter_1_l_re, butter_1_l_im, rom16_re, rom16_im, mul_1_l_re, mul_1_l_im);

    shift_16#(8, 13) shift_2(clk, rst_n, mul_1_l_re, mul_1_l_im, state1_shift_2_re, state1_shift_2_im);

    //---------State2-------------

    commutator_input #(13) com2 (1'b0, flag_in_com1, flag_in_com2, flag_switch_state2_1, mul_1_up_re, mul_1_up_im, state1_shift_2_re, state1_shift_2_im, MDCOutUpRe, MDCOutUpIm, MDCOutDownRe, MDCOutDownIm);


endmodule